## Importing libraries

Baisc

In [1]:
import time
import random
import numpy as np
import pandas as pd
from statistics import mode 
from tqdm.notebook import tqdm

Torch

In [2]:
# From torch
import torch
## nn
import torch.nn as nn
import torch.nn.functional as F
## optim
import torch.optim as optim
from torch.optim import lr_scheduler
## utils
from torch.utils.data import random_split
from torch.utils.data.dataset import Dataset
## torchvision
import torchvision
from torchvision import datasets, models, transforms

SKLearn

In [3]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Loading and transformation

#### General

In [4]:
cls2id = {"Happy": 0, "Sad": 1, "Fear": 2}
id2cls = ["Happy", "Sad", "Fear"]

BATCHSIZE = 50
PATH = "aithon2020_level2_traning.csv"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Defining functions

In [5]:
def load_data(PATH):
    data     = pd.read_csv(PATH)
    try:
        labels   = data["emotion"]
        data     = data.drop(["emotion"], axis = 1)
    except:
        labels = None
    images   = np.array(data.values).reshape(len(data.values), 48, 48)
    images   = images/255
    return images, labels
    
def loader(PATH):
    images, labels = load_data(PATH)
    images = torch.tensor(images)
    images = images.view(images.shape[0], -1, images.shape[1], images.shape[2])

    if labels is not None:
        target = []
        for label in labels.values:
            target.append(cls2id[label])
        target = torch.tensor(target)
    else:
        target = None
    
    return images, target

def data_split(X, Y, test_size, shuffle = True):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test_size, shuffle = shuffle)
    return(X_train, X_test, Y_train, Y_test)

def create_batch(X, Y, batch_size = 1):
    batch_x = [X[i: i + batch_size] for i in range(0, len(X), batch_size)]
    batch_y = [Y[i: i + batch_size] for i in range(0, len(Y), batch_size)] 
    return list(zip(batch_x, batch_y))

def flatten_array(X):
    X = X.reshape(X.shape[0], X.shape[2] * X.shape[3])
    return X

def ML_compatible(x_train, y_train, x_test = None, y_test = None):
    x_train = flatten_array(x_train)
    x_train = x_train.numpy()
    y_train = y_train.numpy()
    if x_test is not None:
            x_test = flatten_array(x_test)
            x_test = x_test.numpy()
            y_test = y_test.numpy()
    return x_train, y_train, x_test, y_test

def oversampling(x_train, y_train):
    dic = {}
    for i in range(len(y_train)):
        try:
            dic[y_train[i].tolist()].append(x_train[i])
        except:  
            dic[y_train[i].tolist()] = [x_train[i]]

    max_occurance = max([len(dic[key]) for key in dic.keys()])

    for key in dic.keys():
        l = (max_occurance - len(dic[key]))
        while l > 0:
            x_train = torch.cat((x_train, random.choice(dic[key]).view(-1, 1, 48, 48)))
            y_train = torch.cat((y_train, torch.tensor([key])))
            l -= 1
            
    return x_train, y_train

#### Main

In [16]:
# train_length      = round(0.7 * datalen)
# test_length       = round(0.3 * datalen)

## Loading images
images, target = loader(PATH)
## Train test split
train_X, test_X, train_Y, test_Y = data_split(images, target, test_size = 0.3)
## Oversampling
train_X, train_Y = oversampling(train_X, train_Y)
## Train loader 
trainloader = create_batch(train_X, train_Y, batch_size = BATCHSIZE)
## Test loader
testloader = create_batch(test_X, test_Y, batch_size = BATCHSIZE)

In [7]:
train_X_ML , train_Y_ML, test_X_ML, test_Y_ML = ML_compatible(train_X, train_Y, test_X, test_Y)

## Models

Used 3 types of model

    1. RESNET101
    2. VGG19
    3. XGBOOST

Finally used bagging to ensemble those models

#### Defining functions

Gradient freezer

1. RESNET101

In [8]:
class RESNET(nn.Module):
    def __init__(self, criterion = None, optimizer = None, learning_rate = 0.001, image_dimention = 1, categories = 3):
        super(RESNET, self).__init__()
        ## Defining networt
         # Defaulf input image dimention is 1
         # Default output categories is 3
        self.pretrained = models.resnext101_32x8d(pretrained = True)
        self.pretrained.conv1 = nn.Conv2d(image_dimention, 64, kernel_size = (3, 3), stride=(2,2), padding=(3,3), bias=False)
        num_ftrs = self.pretrained.fc.in_features
        self.pretrained.fc = nn.Linear(num_ftrs, categories)
        
        ## Defining optimizer and loss function
         # Default loss function is cross entropy
         # Default optimizer is SGD
         # Default learning rate is 0.001
        if criterion:
            self.criterion = criterion
        else:
            self.criterion = nn.CrossEntropyLoss()
        if optimizer:
            self.optimizer = optimizer
        else:
            self.optimizer = optim.SGD(self.pretrained.parameters(), lr = learning_rate, momentum = 0.9)
        
    def forward(self, x):
        x = self.pretrained.forward(x)
        return x
        
    def train(self, traindata, valdata = None, numberEpoch = 10, DEBUG = True):
        trainlen = sum(list(batch[0].shape[0] for batch in traindata))
        total_batch = len(traindata)
        ## Loop over the dataset multiple times
        for epoch in range(numberEpoch): 
            running_corrects = 0.0
            running_loss     = 0.0
            if DEBUG:
                pbar = tqdm(enumerate(traindata, 0), total = total_batch, desc = "Loss 0, Completed", ncols = 800)
            else:
                pbar = enumerate(traindata, 0)
            for count, data in pbar:
                inputs, labels = data[0], data[1]
                inputs = inputs.type(torch.FloatTensor)
                inputs, labels = inputs.to(device), labels.to(device)
                batch  = inputs.shape[0]
                
                ## zero the parameter gradients
                self.optimizer.zero_grad()
                
                ## forward + backward + optimize
                outputs = self.forward(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                
                ## Calculating statistics
                running_loss += loss.item() * batch
                running_corrects += torch.sum(preds == labels.data)
                
                ## Showing statistics
                if DEBUG:
                    pbar.set_description("Loss %.3f, Completed" %(running_loss/trainlen))
                    
            if DEBUG:
                epoch_loss = running_loss/trainlen
                epoch_acc  = running_corrects/trainlen
                print('Epoch %d completed, average loss: %.3f, accuracy: %.3f' %(epoch + 1, epoch_loss, epoch_acc))
            
                if valdata:
                    val_loss, val_acc = self.evaluate(valdata)
                    print('Validation, average loss: %.3f, accuracy: %.3f' %(val_loss, val_acc))
                
    def evaluate(self, testdata):
        running_corrects = 0.0
        running_loss     = 0.0
        testlen = sum(list(batch[0].shape[0] for batch in testdata))
        for data in testdata:
            inputs, labels = data[0], data[1]
            inputs = inputs.type(torch.FloatTensor)
            inputs, labels = inputs.to(device), labels.to(device)
            batch  = inputs.shape[0]          
            ## Forward
            outputs = self.forward(inputs)
            _, preds = torch.max(outputs, 1)
            ## Loss and accuracy
            loss = self.criterion(outputs, labels)
            running_loss += loss.item() * batch
            running_corrects += torch.sum(preds == labels.data)
            
        loss = running_loss/testlen
        acc  = running_corrects/testlen
        return loss, acc
        
    def predict(self, testdata, ID = None):
        predicted_labels = []
        for data in testdata:
            inputs, labels = data[0], data[1]
            inputs = inputs.type(torch.FloatTensor)
            inputs, labels = inputs.to(device), labels.to(device)
            batch  = inputs.shape[0]          
            ## Forward
            outputs = self.forward(inputs)
            _, preds = torch.max(outputs, 1)
            predicted_labels += preds.tolist()
        if ID:
            return([ID[label] for label in predicted_labels])
        return predicted_labels

2. VGG19

In [8]:
class VGGNET(nn.Module):
    def __init__(self, criterion = None, optimizer = None, learning_rate = 0.001, image_dimention = 1, categories = 3):
        super(VGGNET, self).__init__()
        ## Defining networt
         # Defaulf input image dimention is 1
         # Default output categories is 3
        self.pretrained = models.vgg16(pretrained = True)
        self.pretrained.features[0] = nn.Conv2d(image_dimention, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        num_ftrs = self.pretrained.classifier[6].in_features
        self.pretrained.classifier[6] = nn.Linear(num_ftrs, categories)
        
        ## Defining optimizer and loss function
         # Default loss function is cross entropy
         # Default optimizer is SGD
         # Default learning rate is 0.001
        if criterion:
            self.criterion = criterion
        else:
            self.criterion = nn.CrossEntropyLoss()
        if optimizer:
            self.optimizer = optimizer
        else:
            self.optimizer = optim.SGD(self.pretrained.parameters(), lr = learning_rate, momentum = 0.9)
#         self.scheduler = lr_scheduler.StepLR(self.optimizer, step_size = 3, gamma = 0.1)
        
    def forward(self, x):
        x = self.pretrained.forward(x)
        return x
        
    def train(self, traindata, valdata = None, numberEpoch = 10, DEBUG = True):
        
        trainlen = sum(list(batch[0].shape[0] for batch in traindata))
        total_batch = len(traindata)
        ## Loop over the dataset multiple times
        for epoch in range(numberEpoch): 
            running_corrects = 0.0
            running_loss     = 0.0
            if DEBUG:
                pbar = tqdm(enumerate(traindata, 0), total = total_batch, desc = "Loss 0, Completed", ncols = 800)
            else:
                pbar = enumerate(traindata, 0)
            for count, data in pbar:
                inputs, labels = data[0], data[1]
                inputs = inputs.type(torch.FloatTensor)
                inputs, labels = inputs.to(device), labels.to(device)
                batch  = inputs.shape[0]
                
                ## zero the parameter gradients
                self.optimizer.zero_grad()
                
                ## forward + backward + optimize
                outputs = self.forward(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                
                ## Calculating statistics
                running_loss += loss.item() * batch
                running_corrects += torch.sum(preds == labels.data)
                
                ## Learning rate scheduler
#                 self.scheduler.step()
                
                ## Showing statistics
                if DEBUG:
                    pbar.set_description("Loss %.3f, Completed" %(running_loss/trainlen))
            if DEBUG:
                epoch_loss = running_loss/trainlen
                epoch_acc  = running_corrects/trainlen
                print('Epoch %d completed, average loss: %.3f, accuracy: %.3f' %(epoch + 1, epoch_loss, epoch_acc))
            
                if valdata:
                    val_loss, val_acc = self.evaluate(valdata)
                    print('Validation, average loss: %.3f, accuracy: %.3f' %(val_loss, val_acc))
                
    def evaluate(self, testdata):
        running_corrects = 0.0
        running_loss     = 0.0
        testlen = sum(list(batch[0].shape[0] for batch in testdata))
        with torch.no_grad():
            for data in testdata:
                inputs, labels = data[0], data[1]
                inputs = inputs.type(torch.FloatTensor)
                inputs, labels = inputs.to(device), labels.to(device)
                batch  = inputs.shape[0]
                ## Forward
                outputs = self.forward(inputs)
                _, preds = torch.max(outputs, 1)
                ## Loss and accuracy
                loss = self.criterion(outputs, labels)
                running_loss += loss.item() * batch
                running_corrects += torch.sum(preds == labels.data)
            
        loss = running_loss/testlen
        acc  = running_corrects/testlen
        return loss, acc
        
    def predict(self, testdata, ID = None):
        predicted_labels = []
        for data in testdata:
            inputs, labels = data[0], data[1]
            inputs = inputs.type(torch.FloatTensor)
            inputs, labels = inputs.to(device), labels.to(device)
            batch  = inputs.shape[0]           
            ## Forward
            outputs = self.forward(inputs)
            _, preds = torch.max(outputs, 1)
            predicted_labels += preds.tolist()
        if ID:
            return([ID[label] for label in predicted_labels])
        return predicted_labels

3. XGBOOST

In [10]:
model_XGB = XGBClassifier(max_depth = 3000)

## Training

RESNET

In [11]:
model_resnet = RESNET()
model_resnet = model_resnet.to(device)

In [12]:
model_resnet.train(trainloader, valdata = testloader, numberEpoch = 30, DEBUG = False)

VGG19

In [9]:
model_vgg = VGGNET()
model_vgg = model_vgg.to(device)

In [11]:
# model_vgg.train(trainloader, valdata = testloader, numberEpoch = 50, DEBUG = True)

XGBoost

In [16]:
model_XGB.fit(train_X_ML, train_Y_ML)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3000,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Prediction and bagging

In [20]:
## Resnet
prediction_resnet = model_resnet.predict(testloader)
## VGG
prediction_vgg    = model_vgg.predict(testloader)
# XGB
prediction_xgb    = model_XGB.predict(test_X_ML)

In [39]:
def bagged_prediction(prediction_list):
    combined_prediction = list(zip(*prediction_list))
    final_prediction   = []
    for pred in combined_prediction:
        try:
            final_prediction.append(mode(pred))
        except:
            final_prediction.append(pred[1])
    return final_prediction

In [40]:
final_prediction = bagged_prediction([prediction_resnet, prediction_vgg, prediction_xgb])

## Rough

In [21]:
accuracy_score(prediction_resnet, test_Y)

0.6617375231053605

In [22]:
accuracy_score(prediction_vgg, test_Y)

0.7045594577942083

In [23]:
accuracy_score(prediction_xgb, test_Y_ML)

0.61090573012939

In [41]:
accuracy_score(final_prediction, test_Y_ML)

0.7097966728280961

In [19]:
model_resnet.evaluate(trainloader)

(8.29505995454485e-05, tensor(1., device='cuda:0'))

In [43]:
model_vgg.evaluate(trainloader)

(0.14051205685305387, tensor(0.9472))